# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-20 12:56:11,617 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
! hadoop fs -copyFromLocal clickstream.csv /

! hadoop fs -ls -h /clickstream.csv

copyFromLocal: `/clickstream.csv': File exists
-rw-r--r--   1 jovyan supergroup     30.7 M 2024-01-18 18:18 /clickstream.csv


In [3]:
click = se.read.options(delimiter="\t").csv("hdfs:////clickstream.csv", header=True)
click.registerTempTable("click")

In [4]:
click.show(1)

+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|    562|       507|      page|      main|1695584127|
+-------+----------+----------+----------+----------+
only showing top 1 row



### Spark SQL

In [89]:
se.sql(
    '''
    SELECT 
        route,
        COUNT(distinct user_id, session_id) as count
    FROM (
        SELECT
            user_id,
            session_id,
            concat_ws("-", collect_list(event_page)) as route
        FROM (
            SELECT 
                user_id,
                session_id,
                event_page,
                MIN_BY(timestamp, ri) as timestamp,
                MIN(ri) as event_page_appearance
            FROM (
                SELECT
                    user_id,
                    session_id,
                    event_type,
                    event_page,
                    timestamp,
                    ri
                FROM (
                    SELECT distinct
                        user_id,
                        session_id,
                        event_type,
                        event_page,
                        timestamp,
                        ri,
                        SUM(is_error) OVER (partition by user_id, session_id ORDER BY ri) as is_error
                    FROM (
                        SELECT 
                            distinct
                            *,
                            cast(lower(event_type) LIKE '%error%' as int) as is_error,
                            monotonically_increasing_id() as ri
                        FROM click
                        )
                    )
                WHERE is_error = 0
                )
            GROUP BY 
                user_id,
                session_id,
                event_page
            ORDER BY event_page_appearance
            )
        GROUP BY 
            user_id,
            session_id
        )
    GROUP BY route
    ORDER BY count DESC
    ''').show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8185|
|        main-archive| 1213|
|         main-rabota| 1155|
|       main-internet|  977|
|          main-bonus|  940|
|           main-news|  832|
|        main-tariffs|  733|
|         main-online|  635|
|          main-vklad|  548|
| main-archive-rabota|  232|
| main-rabota-archive|  224|
|main-internet-rabota|  188|
|  main-bonus-archive|  185|
|main-archive-inte...|  185|
|   main-bonus-rabota|  185|
|    main-news-rabota|  181|
|   main-rabota-bonus|  179|
|main-rabota-internet|  177|
|  main-archive-bonus|  172|
|   main-archive-news|  171|
|main-internet-arc...|  171|
|    main-rabota-news|  167|
|   main-news-archive|  153|
| main-internet-bonus|  152|
|main-archive-tariffs|  146|
|  main-internet-news|  139|
|  main-news-internet|  134|
| main-archive-online|  133|
|main-tariffs-archive|  132|
|main-tariffs-inte...|  128|
+--------------------+-----+
only showing t

### Spark df

In [102]:
from pyspark.sql.functions import sum, col, lower, monotonically_increasing_id, min
from pyspark.sql.functions import concat_ws, collect_list, countDistinct, desc
from pyspark.sql.window import Window


In [80]:
click_df = se.table("click")

In [81]:
click_df = click_df.withColumn("is_error", (lower(col("event_type")).like("%error%")).cast("int"))
click_df = click_df.withColumn("ri", monotonically_increasing_id())
click_df = click_df.dropDuplicates()

In [82]:
selected_columns = ["user_id", "session_id", "event_type", "event_page", "timestamp", "ri", "is_error"]
df = click_df.select(*selected_columns)

df = df.withColumn("is_error", sum("is_error").over(
    Window.partitionBy("user_id", "session_id").orderBy("ri")
))

In [87]:
filtered_df = df.filter(df.is_error == 0)

In [95]:
aggregated_df = filtered_df.groupBy("user_id", "session_id", "event_page").agg(
    min("ri").alias("event_page_appearance")
).orderBy("event_page_appearance")

In [98]:
grouped_df = aggregated_df.groupBy("user_id", "session_id").agg(
    concat_ws("-", collect_list("event_page")).alias("route")
)

In [103]:
counted_df = grouped_df.groupBy("route").agg(
    countDistinct("user_id", "session_id").alias("count")
).orderBy(desc("count"))

In [104]:
counted_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8185|
|        main-archive| 1212|
|         main-rabota| 1154|
|       main-internet|  974|
|          main-bonus|  939|
|           main-news|  832|
|        main-tariffs|  731|
|         main-online|  635|
|          main-vklad|  547|
| main-archive-rabota|  232|
| main-rabota-archive|  223|
|main-internet-rabota|  187|
|  main-bonus-archive|  185|
|main-archive-inte...|  185|
|   main-bonus-rabota|  185|
|    main-news-rabota|  181|
|   main-rabota-bonus|  179|
|main-rabota-internet|  176|
|   main-archive-news|  171|
|  main-archive-bonus|  171|
|main-internet-arc...|  171|
|    main-rabota-news|  168|
|   main-news-archive|  153|
| main-internet-bonus|  150|
|main-archive-tariffs|  146|
|  main-internet-news|  140|
|  main-news-internet|  134|
| main-archive-online|  133|
|main-tariffs-archive|  132|
|main-tariffs-inte...|  128|
+--------------------+-----+
only showing t

In [128]:
df = click_df.withColumn("is_error", (lower(col("event_type")).like("%error%")).cast("int"))
df = df.withColumn("ri", monotonically_increasing_id())
df = df.dropDuplicates()

selected_columns = ["user_id", "session_id", "event_type", "event_page", "timestamp", "ri", "is_error"]
df = df.select(*selected_columns)

df = df.withColumn("is_error", sum("is_error").over(
    Window.partitionBy("user_id", "session_id").orderBy("ri")
))

df = df.filter(df.is_error == 0)

df = df.groupBy("user_id", "session_id", "event_page").agg(
    min("ri").alias("event_page_appearance")
).orderBy("event_page_appearance")

df = df.groupBy("user_id", "session_id").agg(
    concat_ws("-", collect_list("event_page")).alias("route")
)

df = df.groupBy("route").agg(
    countDistinct("user_id", "session_id").alias("count")
).orderBy(desc("count"))

df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8185|
|        main-archive| 1213|
|         main-rabota| 1154|
|       main-internet|  979|
|          main-bonus|  941|
|           main-news|  831|
|        main-tariffs|  732|
|         main-online|  635|
|          main-vklad|  548|
| main-archive-rabota|  232|
| main-rabota-archive|  223|
|main-internet-rabota|  188|
|  main-bonus-archive|  185|
|main-archive-inte...|  185|
|   main-bonus-rabota|  185|
|    main-news-rabota|  181|
|   main-rabota-bonus|  179|
|main-rabota-internet|  176|
|  main-archive-bonus|  172|
|main-internet-arc...|  172|
|   main-archive-news|  171|
|    main-rabota-news|  168|
|   main-news-archive|  153|
| main-internet-bonus|  151|
|main-archive-tariffs|  146|
|  main-internet-news|  140|
| main-archive-online|  134|
|  main-news-internet|  134|
|main-tariffs-archive|  132|
|main-tariffs-inte...|  128|
+--------------------+-----+
only showing t